In [64]:
import pandas
from geopy.geocoders import Nominatim
nom = Nominatim(scheme='http')


try:
    import json
except ImportError:
    import simplejson as json
    
from twitter import Twitter, OAuth, TwitterHTTPError, TwitterStream


In [65]:
ACCESS_TOKEN    = '3316260091-TxVTwgvv4MgMh1TuqCBhuWBt8taYKx9f3NTLeRL'
ACCESS_SECRET   = '50nR14aSrB6ExkHnw1GbG7Tm1pcWGQx3BUj9oBGTFOsar'
CONSUMER_KEY    = 'wr3jIsxpv2UvHHRGSuR9FWYF1'
CONSUMER_SECRET = 'GLpGRXwxtvs5VlrJibxgMqSQr78Nbl6kIursecWFIRAgyiA8N2'

oauth = OAuth(ACCESS_TOKEN, ACCESS_SECRET, CONSUMER_KEY, CONSUMER_SECRET)

In [66]:
# Initiate the connection to Twitter Streaming API
twitter_stream = TwitterStream(auth=oauth)
# Get a sample of the public data following through Twitter
#iterator = twitter_stream.statuses.sample()
iterator = twitter_stream.statuses.filter(track="trump",language='en')

In [67]:
# Print each tweet in the stream to the screen 
# Here we set it to stop after getting 1000 tweets. 
# You don't have to set it to stop, but can continue running 
# the Twitter API to collect data for days or even longer.

def getTweets(num):
    tweet_count = num
    for tweet in iterator:
        tweet_count-=1
        # Twitter Python Tool wraps the data returned by Twitter 
        # as a TwitterDictResponse object.
        # We convert it back to the JSON format to print/score


        # The command below will do pretty printing for JSON data, try it out
        # print json.dumps(tweet, indent=4)
        data = json.dumps(tweet)
        
        if tweet_count <= 0:
            break
    return data
tweets = getTweets(100) 

tweets_filename = "tweets.txt"
with open(tweets_filename, "w+") as file:
    file.write(tweets)
        

In [68]:
#open text file containing json data for tweets gathered from previous opperation
tweets_file = open(tweets_filename, 'r')
#num = sum(1 for line in tweets_file)
#print("number of lines= "+ str(num))

for line in tweets_file:
    try:
        #read in each line of text file and convert to json object
        tweet = json.loads(line.strip())
        if 'text' in tweet:
            if 'location' in tweet['user']:
                print(tweet['id'])
                print(tweet['created_at'])
                print(tweet['text'])

                print(tweet['user']['id'])
                print(tweet['user']['name'])
                print(tweet['user']['screen_name'])
                print(tweet['user']['location'])

                hashtags = []
                for hashtag in tweet['entities']['hashtags']:
                    hashtags.append("#")
                    hashtags.append(hashtag['text'])
                print(hashtags)
            else:
                print("there was no text or location in this tweet")
    except:
        #sometimes I guess a line is not in json format
        print("this line was not in json format?")
        continue

920290650871930881
Tue Oct 17 14:09:23 +0000 2017
RT @Newsweek: Donald Trump blames everyone but himself for failed campaign promises, in bizarre statement https://t.co/R7hkQtdfHp https://t…
24678075
LOLS@Trumpsters 🇨🇦
SusanSaysHi
Canada
[]


In [69]:
#open tweets_file saved from api stream
with open(tweets_filename,'r') as tweets_file:
    #get data (location, text) into padas dataframe
    granularity = {
        "location":[],
        "text":[]
    }
    #write data into dictionary defined above
    for line in tweets_file:
        tweet = json.loads(line.strip())
        granularity["location"].append(tweet['user']['location'])
        granularity["text"].append(tweet['text'])  
#convert dictionary to pandas dataframe        
df_granularity = pandas.DataFrame.from_dict(granularity)

#add coordinates column to data frame, geocoded data of location from tweet
df_granularity["geo_location"] = df_granularity["location"].apply(nom.geocode)

#iterate through the geo_location column and return lat/lon if there is a lat/lon.
df_granularity["lat"] = df_granularity.geo_location.apply(lambda x: x.latitude if x != None else None)
df_granularity["lon"] = df_granularity.geo_location.apply(lambda x: x.longitude if x != None else None)

df_granularity
#write df to csv file.

location                                               text  \
0   Canada  RT @Newsweek: Donald Trump blames everyone but...   

                           geo_location        lat         lon  
0  (Canada, (61.0666922, -107.9917071))  61.066692 -107.991707